In [2]:
import hsfs
import boto3
from pyspark.sql import DataFrame
from pyspark.sql.functions import col

In [3]:
spark.conf.set("spark.sql.shuffle.partitions", 2)

In [4]:
connection = hsfs.connection()
fs = connection.get_feature_store()

# Storage connector to s3
sc = fs.get_storage_connector("experiment-s3")


Connected. Call `.close()` to terminate connection gracefully.

In [5]:
sc.prepare_spark()

In [6]:
s3_directory = "s3a://" + sc.bucket + "/axel_experiments"

In [7]:
def save(experiment_id, direction: str, left: DataFrame, right: DataFrame):
    directory = s3_directory + "/sorted-{}/{}".format(direction, experiment_id)
    left.write.parquet(directory + "/left.parquet", mode="overwrite")
    # left.to_csv(directory + "/debug_left.csv")
    right.write.parquet(directory + "/right.parquet", mode="overwrite")
    # right.to_csv(directory + "/debug_right.csv")

In [8]:
s3_raw_directory = "s3a://" + sc.bucket + "/axel_experiments/raw/"
def load_data(experiment_directory: str):
    left_raw = s3_raw_directory + "/{}/left.parquet".format(experiment_directory)
    right_raw = s3_raw_directory + "/{}/right.parquet".format(experiment_directory)
    
    return spark.read.parquet(left_raw), spark.read.parquet(right_raw)

In [9]:
client = boto3.client(
    "s3",
    aws_access_key_id=sc._access_key,
    aws_secret_access_key=sc._secret_key,
)

In [10]:
folders_results = client.list_objects(Bucket=sc.bucket, Prefix="axel_experiments/raw/", Delimiter='/')

In [11]:
experiment_ids = []

for folder in folders_results.get("CommonPrefixes"):
    folder_name = folder.get("Prefix")
    experiment_ids.append(folder_name.split("/")[2])
experiment_ids

['10000-1_year', '100000-1_year', '1000000-1_year', '10000000-1_year']

In [12]:
def sort(df: DataFrame):
    return (
        df.orderBy(col("id"), col("ts")),
        df.orderBy(col("id").desc(), col("ts").desc()),
    )

In [12]:
def handle_experiments(ids):
    for experiment_id in ids:
        left_raw, right_raw = load_data(experiment_id)
        left_asc, left_desc = sort(left_raw)
        right_asc, right_desc = sort(right_raw)
        save(experiment_id, "asc", left_asc, right_asc)
        save(experiment_id, "desc", left_desc, right_desc)
        

In [13]:
handle_experiments(experiment_ids)

# Create randomized collections aswell

In [15]:
from pyspark.sql.functions import rand 

def randomize(df: DataFrame):
    return df.orderBy(rand())

In [21]:
def handle_experiments_random(ids):
    for experiment_id in ids:
        left_raw, right_raw = load_data(experiment_id)
        left_rand = randomize(left_raw)
        right_rand = randomize(right_raw)
        save(experiment_id, "rand", left_rand, right_rand)

In [ ]:
handle_experiments_random(experiment_ids)